# `numpy`中`ndarray`对象的构造, 处理与应用

In [2]:
#以别名形式导入numpy. 
import numpy as np; 
#导入与ndarray有关的数据类型. 
from numpy import int8, int16, int32, int64; 
from numpy import uint8, uint16, uint32, uint64; 
from numpy import float16, float32, float64; 
from numpy import complex64, complex128; 

## `numpy.ndarray`对象的特征
与Python中内置的`list`和`tuple`对象不同, `numpy.array`对象中的结构和其中的每个元素必须满足以下要求: 

* 所有元素必须是**同一个类**的不同实例; 
> 相较之下, 同一个`list`或`tuple`对象中, 不同类型(如`int`和`str`)的元素可以共存. 
* 所有元素所在的类是**定长**的, 因此不同实例在内存中占用的大小是固定的; 
    * 如果**强制使用具有变长类型**实例的`list`或者`tuple`构造`ndarray`, 此时**如未指定`dtype`, 则`dtype`将被设定为`object`**, `ndarray`将在数组中存储变长对象的地址, *成为一个指针数组*. 
* 当`array`对象是多维数组(数组嵌套数组)时, 在任何一级数组中, 被嵌套的数组具有相同的长度. 

因此, `numpy.array`的数据部分, 在内存中可能以顺序结构存储

* 使用下标访问时, 可直接根据下标, 确定目标元素地址相对于基址的偏移量. 
* 功能上相当于部分早期编程语言中(如C, Basic, Pascal, IDL)的`array`, **只能存储同类型数据**; 
* 在大批量检索和修改个别数据的场合, 其速度比python中内置的`list`更快. 

## `numpy.ndarray`对象的构造

### 将`list`或`tuple`转换为`numpy.ndarray`
用法: 
```python
numpy.array(seq, dtype = dt)
```
* `seq` 列表(`list`对象)或元组(`tuple`对象)
* `seq`结构要求: 
    * 当`seq`存在嵌套的`list`或`tuple`时, `seq`的所有元素必须等长; 
    * 当前述嵌套出现二层以上时, 要求嵌套的每层元素均满足上一要求; 
* `dtype = dt` 数组内各元素的类型和字长
* 支持的类型

|表示范围|类型名称(需使用`from numpy import ...`导入为当前上下文中的名称)|
|:-:|:-|
|有符号整数|`int8`, `int16`, `int32`, `int64`|
|无符号整数|`uint8`, `uint16`, `uint32`, `uint64`|
|浮点小数|`float16`, `float32`, `float64`|
|复数|`complex64`, `complex128`|

### 利用`numpy`函数构造特殊数组

In [37]:
#定义二维旋转变换矩阵(绕原点旋转)
def rot2D_matrix(rot_angle): 
    cθ = np.cos(rot_angle); sθ = np.sin(rot_angle); 
    ls = [[cθ, -sθ, 0], [sθ, cθ, 0], [0, 0, 1]]; 
    return(np.array(ls, dtype = numpy.float64)); 

In [122]:
matA = rot2D_matrix(np.pi / 2); 
print(matA.ndim, matA.size, matA.shape, matA.dtype); 

2 9 (3, 3) float64
